<a href="https://colab.research.google.com/github/shilpathota/AI_ML/blob/master/KidsStoryTeller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Dependencies

In [7]:
!pip install langchain langchain-community huggingface_hub --quiet

In [13]:
!pip install langgraph --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.9 MB/s eta 0:00:00


Setting up hugging face Mistral LLM

In [124]:
from huggingface_hub import InferenceClient
import os
import getpass

# 🔐 Replace with your Hugging Face token from https://huggingface.co/settings/tokens
HUGGINGFACE_API_KEY = getpass.getpass("HUGGINGFACE_API_KEY")
client = InferenceClient(model="HuggingFaceH4/zephyr-7b-beta", token=HUGGINGFACE_API_KEY)

def call_mistral(prompt, temperature=0.7, top_p=0.95):
    try:
        result = client.text_generation(
            prompt=prompt,
            max_new_tokens=500,
            stream=False,
            temperature=temperature,
            top_p=top_p,
            do_sample=True  # 👈 ensures variation
        )
        return result if isinstance(result, str) else "".join(result)
    except Exception as e:
        return f"[ERROR] Failed to generate story: {e}"



HUGGINGFACE_API_KEY··········


In [163]:
from pydantic import BaseModel
from typing import Optional

class StoryState(BaseModel):
    profile: Optional[dict] = None
    base_story: Optional[str] = None
    final_story: Optional[str] = None

Define Agent Tools – Profile, Story, Quiz, Feedback, Coordinator

In [165]:
import json

def node_load_profile(state: StoryState) -> StoryState:
    try:
        with open("profile.json", "r") as f:
            state.profile = json.load(f)
    except:
        state.profile = {"name": "Chetu", "age": 6, "theme": "Paw Patrol", "math_level": "addition"}
    return state

# Node: Story generation
def node_story_agent(state: StoryState) -> StoryState:
    profile = state.profile
    prompt = f"""
    Write a fun math story for a {profile['age']}-year-old named {profile['name']} who loves {profile['theme']}.
    Focus on teaching {profile['math_level']}.
    """
    state.base_story = call_mistral(prompt)
    return state

# Node: Quiz generation
def node_quiz_agent(state: StoryState) -> StoryState:
    story = state.base_story
    profile = state.profile
    prompt = f"""
    Add 2 simple math questions into this story for a child (age {profile['age']}).
    Return the story with questions inline, using this format:
    Q: <question>
    a) ...
    b) ...
    c) ...
    Answer: <answer>

    Story:
    {story}
    """
    state.final_story = call_mistral(prompt)
    return state

def node_feedback(state):
    feedback = state.get("feedback", {})
    profile = state.get("profile", {})
    profile.update(feedback)
    save_profile(profile)
    return state


Initialize LAngChain Agents

In [166]:
from langgraph.graph import StateGraph

builder = StateGraph(StoryState)

builder.add_node("load_profile", node_load_profile)
builder.add_node("story_agent", node_story_agent)
builder.add_node("quiz_agent", node_quiz_agent)

builder.set_entry_point("load_profile")
builder.add_edge("load_profile", "story_agent")
builder.add_edge("story_agent", "quiz_agent")

graph = builder.compile()

Run the flow

In [167]:
result = graph.invoke({})
print("🔁 Final Story with Quiz:\n")
print(result)


🔁 Final Story with Quiz:

{'profile': {'name': 'Chetu', 'age': 6, 'theme': 'Peppa pig', 'math_level': 'addition'}, 'base_story': "[ERROR] OpenAI call failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}", 'final_story': "[ERROR] OpenAI call failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}"}


Integrating UI

In [51]:
!pip install gradio --quiet


In [130]:
import gradio as gr

def run_story_generator(name, age, theme, math_level):
    # Save profile to JSON so LangGraph picks it up
    profile = {"name": name, "age": age, "theme": theme, "math_level": math_level}
    with open("profile.json", "w") as f:
        json.dump(profile, f)

    result = graph.invoke({})  # Call your compiled LangGraph graph
    return result["base_story"]+result["final_story"]


In [131]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧮 Personalized Math Story Generator")

    with gr.Row():
        name = gr.Textbox(label="Child's Name", value="Chetu")
        age = gr.Number(label="Age", value=6)
    theme = gr.Textbox(label="Favorite Theme", value="Paw Patrol")
    math_level = gr.Dropdown(choices=["counting", "addition", "subtraction", "multiplication"], label="Math Topic", value="addition")

    generate_btn = gr.Button("📖 Generate Story")

    output = gr.Textbox(label="Generated Story with Quiz", lines=15)

    generate_btn.click(fn=run_story_generator, inputs=[name, age, theme, math_level], outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5a83d1ec1cf34ea2d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
